In [17]:
import operator
from math import log

In [18]:
#生成数据函数
def creatdataset():
    dataset=[[1,1,'yes'],
             [1,1,'yes'],
             [1,0,'no'],
             [0,1,'no'],
             [0,1,'no']]
    labels=["no surfacing","flippers"]
    return dataset,labels

In [19]:
dataset,labels=creatdataset()
print dataset,labels
print len(dataset)
print len(labels)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']] ['no surfacing', 'flippers']
5
2


In [20]:
#创建计算香浓熵的函数
def calcShannonEnt(dataset):
    classcount={}
    for featvec in dataset:
        currentlabel = featvec[-1]
        if currentlabel not in classcount.keys():
            classcount[currentlabel]=0
        classcount[currentlabel]+=1
    ShannonEnt=0.0
    for each in classcount:   #这里的each是classcount的键
        p = classcount[each]/float(len(dataset))
        ShannonEnt-=p*log(p,2)
    return ShannonEnt            

In [21]:
#测试信息熵函数
ShannonEnt=calcShannonEnt(dataset)
print ShannonEnt

0.970950594455


In [22]:
#按条件划分数据集
def splitDataset(dataset,axis,value):   #要划分的数据集 数据集的第几个特征 按特征值的值进行划分
    retdataset=[]
    for featvec in dataset:
        if featvec[axis] == value:
            #如果符合要求，将所有符合要求的数据提取出来并减去该特征值对应的数据列
            reducedfeatvec = featvec[:axis]
            reducedfeatvec.extend(featvec[axis+1:])
            retdataset.append(reducedfeatvec)
    return retdataset
    

In [23]:
#测试划分数据集的效果
retdataset=splitDataset(dataset,0,1)
retdataset

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [24]:
#选择最好的特征值进行数据分割
#按不同的特征值和特征值的值进行分割然后计算信息熵。进行排序。
def ChooseBestFeatureToSplit(dataset):
    numfeatures=len(dataset[0])-1   #共有几个特征属性
    #计算分割数据之前的信息熵
    bestFeature = -1
    bestinfogain=0.0
    basenonEnt=calcShannonEnt(dataset)
    for i in range(numfeatures):
        newEntropy=0.0
        #遍历所有行收集所有的取值然后去重
        featlist=[example[i] for example in dataset]
        uniquevals=set(featlist)  #去重
        for value in uniquevals:
            #按特征属性和属性值分割数据集
            retmat=splitDataset(dataset,i,value)
            #计算新数据集占总数据集的权重
            prob=len(retmat)/float(len(dataset))
            newEntropy+=prob*calcShannonEnt(retmat)
        infogain=basenonEnt-newEntropy  #计算减少的信息熵
        if infogain>bestinfogain:
            bestinfogain=infogain
            bestfeature = i
    return bestfeature  

In [25]:
#检验上函数的正确性
dataset,labels=creatdataset()
bestfeature=ChooseBestFeatureToSplit(dataset)
print bestfeature

0


In [26]:
#当消耗完所有的属性时，数据集的标签依然没有统一，这时采用多数决定的方法。
def majorityCnt(classlist):
    classCount = {}
    for each in classlist:
        if each not in classCount:
            classCount[each]=0
        classCount[each]+=1
    sortedClassCount=sorted(classCount.iteritems(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]
            

In [27]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList): 
        return classList[0]#stop splitting when all of the classes are equal
    if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet
        return majorityCnt(classList)
    bestFeat = ChooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataset(dataSet, bestFeat, value),subLabels)
    return myTree          

In [28]:
tree=createTree(dataset,labels)

In [30]:
print tree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}


In [31]:
#创建分类器
def classify(inputTree,featLabels,testVec):  #参数分别为以字典形式组织的决策树，特征列表和要分类的特征向量
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): 
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel

In [39]:
data,label=creatdataset()
print data
print label

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
['no surfacing', 'flippers']


In [41]:
classLabel=classify(tree,label,[1,0])
print classLabel

no


In [42]:
#决策树的存储 持久化存储
def storeTree(inputTree,filename):
    import pickle
    fw = open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()

In [43]:
#将磁盘文件解析为python对象
def grabTree(filename):
    import pickle
    fr = open(filename)
    return pickle.load(fr)

In [45]:
#测试存取决策树的函数
storeTree(tree,"mytree.txt")

In [46]:
grabTree("mytree.txt")

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [49]:
#用决策树算法预测隐形眼睛的镜片类型
f=open("lenses.txt","r")
buf=f.readlines()
lenses_list=[]
for example in buf:
    li=example.strip().split("\t")
    lenses_list.append(li)

In [50]:
print lenses_list

[['young', 'myope', 'no', 'reduced', 'no lenses'], ['young', 'myope', 'no', 'normal', 'soft'], ['young', 'myope', 'yes', 'reduced', 'no lenses'], ['young', 'myope', 'yes', 'normal', 'hard'], ['young', 'hyper', 'no', 'reduced', 'no lenses'], ['young', 'hyper', 'no', 'normal', 'soft'], ['young', 'hyper', 'yes', 'reduced', 'no lenses'], ['young', 'hyper', 'yes', 'normal', 'hard'], ['pre', 'myope', 'no', 'reduced', 'no lenses'], ['pre', 'myope', 'no', 'normal', 'soft'], ['pre', 'myope', 'yes', 'reduced', 'no lenses'], ['pre', 'myope', 'yes', 'normal', 'hard'], ['pre', 'hyper', 'no', 'reduced', 'no lenses'], ['pre', 'hyper', 'no', 'normal', 'soft'], ['pre', 'hyper', 'yes', 'reduced', 'no lenses'], ['pre', 'hyper', 'yes', 'normal', 'no lenses'], ['presbyopic', 'myope', 'no', 'reduced', 'no lenses'], ['presbyopic', 'myope', 'no', 'normal', 'no lenses'], ['presbyopic', 'myope', 'yes', 'reduced', 'no lenses'], ['presbyopic', 'myope', 'yes', 'normal', 'hard'], ['presbyopic', 'hyper', 'no', 'redu

In [53]:
lab=["age","prescript","astigmatic","tearRate"]
#创建决策树
mytree=createTree(lenses_list,lab)

In [54]:
print mytree

{'tearRate': {'reduced': 'no lenses', 'normal': {'astigmatic': {'yes': {'prescript': {'hyper': {'age': {'pre': 'no lenses', 'presbyopic': 'no lenses', 'young': 'hard'}}, 'myope': 'hard'}}, 'no': {'age': {'pre': 'soft', 'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}}, 'young': 'soft'}}}}}}
